# Using ANN to predict if a customer will leave the bank?

In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
import os
%cd ..
%cd dataset

D:\ds\portfolio
D:\ds\portfolio\dataset


In [3]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')

In [4]:
%cd ..
%cd bankprediction

D:\ds\portfolio
D:\ds\portfolio\bankprediction


In [5]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
dataset['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [7]:
dataset.shape

(10000, 14)

In [8]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [9]:
X.shape

(10000, 10)

In [10]:
X[0]

array([619, 'France', 'Female', 42, 2, 0.0, 1, 1, 1, 101348.88], dtype=object)

In [11]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]

In [12]:
X[1].astype(object)

array([0.0, 1.0, 608.0, 0.0, 41.0, 1.0, 83807.86, 1.0, 0.0, 1.0, 112542.58], dtype=object)

In [13]:
X.shape

(10000, 11)

In [14]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [15]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [16]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

Using TensorFlow backend.


In [17]:
# Making the ANN

# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 20, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
classifier.add(Dropout(0.1))

# Adding the second hidden layer
classifier.add(Dense(units = 20, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dropout(0.1))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

In [18]:
#Saving the empty model
classifier.save_weights('empty.h5')

In [19]:
#Loading the empty model.
#if we want to change the optimizer and see its effect on model from scratch.
classifier.load_weights('empty.h5')

In [20]:
optimizer = 'Adam'

In [21]:
# Compiling the ANN
classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])

In [22]:
# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 50)

Epoch 1/50
8000/8000 [==============================] - 4s - loss: 0.4600 - acc: 0.7960     
Epoch 2/50
8000/8000 [==============================] - 3s - loss: 0.4255 - acc: 0.8098     
Epoch 3/50
8000/8000 [==============================] - 3s - loss: 0.4189 - acc: 0.8261     
Epoch 4/50
8000/8000 [==============================] - 3s - loss: 0.4155 - acc: 0.8314     
Epoch 5/50
8000/8000 [==============================] - 3s - loss: 0.4111 - acc: 0.8321     
Epoch 6/50
8000/8000 [==============================] - 3s - loss: 0.4117 - acc: 0.8305     
Epoch 7/50
8000/8000 [==============================] - 3s - loss: 0.4086 - acc: 0.8336     
Epoch 8/50
8000/8000 [==============================] - 3s - loss: 0.4096 - acc: 0.8326     
Epoch 9/50
8000/8000 [==============================] - 3s - loss: 0.4091 - acc: 0.8323     
Epoch 10/50
8000/8000 [==============================] - 3s - loss: 0.4057 - acc: 0.8341     
Epoch 11/50
8000/8000 [==============================] - 3s - loss: 0

In [23]:
classifier.save_weights('adam10batch50epoch.h5')

In [24]:
# Making predictions and evaluating the model

# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1525,   70],
       [ 211,  194]])

In [25]:
# Predicting a single new observation
"""Predict if the customer with the following informations will leave the bank:
Geography: France
Credit Score: 600
Gender: Male
Age: 40
Tenure: 3
Balance: 60000
Number of Products: 2
Has Credit Card: Yes
Is Active Member: Yes
Estimated Salary: 50000"""
new_prediction = classifier.predict(sc.transform(np.array([[0.0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
new_prediction = (new_prediction > 0.5)
new_prediction

array([[False]], dtype=bool)

## The new customer for which we are predicting will not leave the bank